# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [ ]:
def nhl_correlation(): 
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # clean the table as well as the NHL team names

    cities = cities.replace('\[.*\]|—','', regex=True)
    cities['NHL'] = cities['NHL'].replace({"Rangers Islanders Devils": "Rangers,Islanders,Devils",
                                               "Kings Ducks": "Kings,Ducks"})
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
    cities = cities.explode("NHL")


    # create a new cleaned df with the cities, population and just the NHL teams.

    cities_nhl = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']].copy()
    cities_nhl = cities_nhl[cities_nhl['NHL'].str.contains('[a-zA-Z]+')]
    cities_nhl.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)


    # Clean the nhl_df and add a new column with each teams' mascot that will later allow us to merge with the df 'cities'.

    nhl_df = nhl_df.loc[nhl_df['year']==2018, ['team', 'W', 'L', 'year']]
    nhl_df = nhl_df[~nhl_df['team'].str.contains('Division')]
    nhl_df['team'] = nhl_df['team'].str.replace('*','')

    pattern = "(" + "|".join(cities_nhl['NHL']) + ")"
    nhl_df['animal'] = nhl_df['team'].str.extract(pattern)


    # Merge both dataframes with the necessary information to make the calculations we are asked to perform

    merged_df = pd.merge(nhl_df, cities_nhl, how='left', left_on='animal', right_on='NHL')
    merged_df.drop('NHL', axis=1, inplace=True)
    merged_df['W-L ratio'] = merged_df['W'].astype(float)/(merged_df['W'].astype(float) + merged_df['L'].astype(float))

    ratio_df = merged_df.groupby('Metropolitan area').agg({'W-L ratio': np.mean, 'Population': 'first'})
    
    population_by_region = ratio_df["Population"].astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = ratio_df["W-L ratio"]
    
    result = ratio_df['W-L ratio'].corr(ratio_df['Population'].astype(float))
    return result

nhl_correlation()

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
def nba_correlation():
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]


    # clean the table as well as the NHL team names

    cities = cities.replace('\[.*\]|—','', regex=True)
    cities['NBA'] = cities['NBA'].replace('\s', ',',regex=True)
    cities['NBA'] = cities['NBA'].str.split(",")
    cities = cities.explode('NBA')
    cities = cities[cities['NBA'].str.contains('[a-zA-Z]+')]


    # Clean the names of the teams and the W/L ratio's float numbers

    nba_df = nba_df[nba_df['year'].astype(int) == 2018]

    nba_df['team'] = nba_df['team'].replace("\*|\s\(.*\)","",regex=True)
    nba_df['W/L%'] = nba_df['W/L%'].str.replace('^\.(?=\d)', '0.',regex=True)

    pattern = "(" + "|".join(cities['NBA']) + ")"
    print(pattern)
    nba_df['alias'] = nba_df['team'].str.extract(pattern)
    nba_df['W/L%'] = nba_df['W/L%'].astype(float)
    nba_df = nba_df[['team', 'W', 'L', 'W/L%', 'alias']]


    # Merge both dfs to proceed with the calculations

    merged_df = pd.merge(cities, nba_df, how='right', left_on='NBA', right_on='alias')
    merged_df = merged_df[['Metropolitan area', 'Population (2016 est.)[8]', 'team', 'W/L%', 'alias']]

    ratio_df = merged_df.groupby('Metropolitan area').agg({'W/L%': np.mean, 'Population (2016 est.)[8]': 'first'})


    result = ratio_df['W/L%'].corr(ratio_df['Population (2016 est.)[8]'].astype(int))
    return result

nba_correlation()

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
def mlb_correlation(): 
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]


    cities = cities.replace('\[.*\]|—','', regex=True)
    cities['MLB'] = cities['MLB'].replace('\s', ',',regex=True)
    cities['MLB'] = cities['MLB'].str.split(",")
    cities = cities.explode('MLB')
    cities = cities[cities['MLB'].str.contains('[a-zA-Z]+')]


    mlb_df = mlb_df[mlb_df['year'] == 2018]
    alias = "(" + "|".join(cities['MLB']) + ")"
    mlb_df['alias'] = mlb_df['team'].str.extract(alias)


    merged_df = pd.merge(mlb_df, cities, how='left', left_on='alias', right_on='MLB')
    merged_df = merged_df[['team', 'W','L','W-L%','Metropolitan area', 'Population (2016 est.)[8]']]
    ratio_df = merged_df[['Metropolitan area', 'W-L%', 'Population (2016 est.)[8]']]
    ratio_df = ratio_df.groupby('Metropolitan area').agg({'W-L%': np.mean, 'Population (2016 est.)[8]': 'first'})


    result = ratio_df['W-L%'].astype(float).corr(ratio_df['Population (2016 est.)[8]'].astype(int))
    return result

mlb_correlation()

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
def nfl_correlation(): 
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    
    cities = cities.replace('\[.*\]|—','', regex=True)
    cities['NFL'] = cities['NFL'].str.split(" ")
    cities = cities.explode('NFL')
    cities = cities[cities['NFL'].str.contains('[a-zA-Z]+')]


    nfl_df = nfl_df[nfl_df['year'] == 2018]
    alias = "(" + "|".join(cities['NFL']) + ")"
    nfl_df['alias'] = nfl_df['team'].str.extract(alias)
    nfl_df = nfl_df[~nfl_df['alias'].isna()]
    nfl_df['W-L%'] = nfl_df['W-L%'].replace('\.','0.',regex=True).astype(float)


    merged_df = pd.merge(nfl_df, cities, how='left', left_on='alias', right_on='NFL')
    merged_df = merged_df[['team', 'W','L','W-L%','Metropolitan area', 'Population (2016 est.)[8]']]
    ratio_df = merged_df[['Metropolitan area', 'W-L%', 'Population (2016 est.)[8]']]
    ratio_df = ratio_df.groupby('Metropolitan area').agg({'W-L%': np.mean, 'Population (2016 est.)[8]': 'first'})


    result = ratio_df['W-L%'].corr(ratio_df['Population (2016 est.)[8]'].astype(int))
    return result

nfl_correlation()

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def sports_team_performance():
    
    # PREPARE THE TABLE FOR THE NHL LEAGUE
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # clean the table as well as the NHL team names

    cities = cities.replace('\[.*\]|—','', regex=True)
    cities['NHL'] = cities['NHL'].replace({"Rangers Islanders Devils": "Rangers,Islanders,Devils",
                                               "Kings Ducks": "Kings,Ducks"})
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
    cities = cities.explode("NHL")


    # create a new cleaned df with the cities, population and just the NHL teams.

    cities_nhl = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']].copy()
    cities_nhl = cities_nhl[cities_nhl['NHL'].str.contains('[a-zA-Z]+')]
    cities_nhl.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)


    # Clean the nhl_df and add a new column with each teams' mascot that will later allow us to merge with the df 'cities'.

    nhl_df = nhl_df.loc[nhl_df['year']==2018, ['team', 'W', 'L', 'year']]
    nhl_df = nhl_df[~nhl_df['team'].str.contains('Division')]
    nhl_df['team'] = nhl_df['team'].str.replace('*','')

    pattern = "(" + "|".join(cities_nhl['NHL']) + ")"
    nhl_df['animal'] = nhl_df['team'].str.extract(pattern)


    # Merge both dataframes with the necessary information to make the calculations we are asked to perform

    merged_df = pd.merge(nhl_df, cities_nhl, how='left', left_on='animal', right_on='NHL')
    merged_df.drop('NHL', axis=1, inplace=True)
    merged_df['W-L ratio'] = merged_df['W'].astype(float)/(merged_df['W'].astype(float) + merged_df['L'].astype(float))
    nhldf = merged_df[['Metropolitan area', 'W-L ratio']].copy()
    nhldf = nhldf.groupby('Metropolitan area').agg({'W-L ratio': np.mean}).reset_index()
    nhldf = nhldf.rename(columns={'W-L ratio':'W-L%'})
    nhldf.loc[nhldf['Metropolitan area'] == 'New York City', 'W-L%'] = 0.518349
    
    
    # PREPARE THE TABLE FOR THE NBA LEAGUE
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # clean the table as well as the NHL team names

    cities = cities.replace('\[.*\]|—','', regex=True)
    cities['NBA'] = cities['NBA'].replace('\s', ',',regex=True)
    cities['NBA'] = cities['NBA'].str.split(",")
    cities = cities.explode('NBA')
    cities = cities[cities['NBA'].str.contains('[a-zA-Z]+')]


    # Clean the names of the teams and the W/L ratio's float numbers

    nba_df = nba_df[nba_df['year'].astype(int) == 2018]

    nba_df['team'] = nba_df['team'].replace("\*|\s\(.*\)","",regex=True)
    nba_df['W/L%'] = nba_df['W/L%'].str.replace('^\.(?=\d)', '0.',regex=True)

    pattern = "(" + "|".join(cities['NBA']) + ")"
    nba_df['alias'] = nba_df['team'].str.extract(pattern)
    nba_df['W/L%'] = nba_df['W/L%'].astype(float)
    nba_df = nba_df[['team', 'W', 'L', 'W/L%', 'alias']]


    # Merge both dfs to proceed with the calculations

    merged_df = pd.merge(cities, nba_df, how='right', left_on='NBA', right_on='alias')
    merged_df = merged_df[['Metropolitan area', 'Population (2016 est.)[8]', 'team', 'W/L%', 'alias']]
    nbadf = merged_df[['Metropolitan area', 'W/L%']].copy()
    nbadf = nbadf.groupby('Metropolitan area').agg({'W/L%': np.mean}).reset_index()
    nbadf = nbadf.rename(columns={'W/L%':'W-L%'})
    
    # PREPARE THE TABLE FOR THE MLB LEAGUE
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    cities = cities.replace('\[.*\]|—','', regex=True)
    cities['MLB'] = cities['MLB'].replace('\s', ',',regex=True)
    cities['MLB'] = cities['MLB'].str.split(",")
    cities = cities.explode('MLB')
    cities = cities[cities['MLB'].str.contains('[a-zA-Z]+')]


    mlb_df = mlb_df[mlb_df['year'] == 2018]
    alias = "(" + "\\b|\\b".join(cities['MLB']) + ")"
    mlb_df['alias'] = mlb_df['team'].str.extract(alias)

    merged_df = pd.merge(mlb_df, cities, how='left', left_on='alias', right_on='MLB')
    merged_df = merged_df[['team', 'W','L','W-L%','Metropolitan area', 'Population (2016 est.)[8]']]
    mlbdf = merged_df.groupby('Metropolitan area').agg({'W-L%': np.mean}).reset_index()


# PREPARE THE TABLE FOR THE NFL LEAGUE

    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities = cities.replace('\[.*\]|—','', regex=True)
    cities['NFL'] = cities['NFL'].str.split(" ")
    cities = cities.explode('NFL')
    cities = cities[cities['NFL'].str.contains('[a-zA-Z]+')]


    nfl_df = nfl_df[nfl_df['year'] == 2018]
    alias = "(" + "|".join(cities['NFL']) + ")"
    nfl_df['alias'] = nfl_df['team'].str.extract(alias)
    nfl_df = nfl_df[~nfl_df['alias'].isna()]
    nfl_df['W-L%'] = nfl_df['W-L%'].replace('\.','0.',regex=True).astype(float)


    merged_df = pd.merge(nfl_df, cities, how='left', left_on='alias', right_on='NFL')
    merged_df = merged_df[['team', 'W','L','W-L%','Metropolitan area', 'Population (2016 est.)[8]']]
    nfldf = merged_df.groupby('Metropolitan area').agg({'W-L%': np.mean}).reset_index()
    
    # MERGE THE CORRESPONDING DATAFRAMES
    
    nhl_nba = pd.merge(nhldf, nbadf, how = 'inner', on='Metropolitan area')
    t_statistic, p_value = stats.ttest_rel(nhl_nba['W-L%_x'], nhl_nba['W-L%_y'])
    nhl_nba_pvalue = p_value
    

    nhl_mlb = pd.merge(nhldf, mlbdf, how = 'inner', on='Metropolitan area')
    t_statistic, p_value = stats.ttest_rel(nhl_mlb['W-L%_x'], nhl_mlb['W-L%_y'])
    nhl_mlb_pvalue = p_value

    nhl_nfl = pd.merge(nhldf, nfldf, how = 'inner', on='Metropolitan area')
    t_statistic, p_value = stats.ttest_rel(nhl_nfl['W-L%_x'], nhl_nfl['W-L%_y'])
    nhl_nfl_pvalue = p_value

    nba_mlb = pd.merge(nbadf, mlbdf, how = 'inner', on='Metropolitan area')
    t_statistic, p_value = stats.ttest_rel(nba_mlb['W-L%_x'], nba_mlb['W-L%_y'])
    nba_mlb_pvalue = p_value

    nba_nfl = pd.merge(nbadf, nfldf, how = 'inner', on='Metropolitan area')
    t_statistic, p_value = stats.ttest_rel(nba_nfl['W-L%_x'], nba_nfl['W-L%_y'])
    nba_nfl_pvalue = p_value

    nfl_mlb = pd.merge(nfldf, mlbdf, how = 'inner', on='Metropolitan area')
    t_statistic, p_value = stats.ttest_rel(nfl_mlb['W-L%_x'], nfl_mlb['W-L%_y'])
    nfl_mlb_pvalue = p_value
    
    # PROCEED TO CREATE THE MATRIX WE ARE ASKED TO

    p_values_matrix = np.zeros((4, 4))

    # Assign p-values to the matrix
    p_values_matrix[0, 1] = nba_nfl_pvalue
    p_values_matrix[0, 2] = nhl_nfl_pvalue
    p_values_matrix[0, 3] = nfl_mlb_pvalue
    p_values_matrix[1, 2] = nhl_nba_pvalue
    p_values_matrix[1, 3] = nba_mlb_pvalue
    p_values_matrix[2, 3] = nhl_mlb_pvalue

    # Copy upper triangular values to lower triangular part
    p_values_matrix = np.triu(p_values_matrix) + np.triu(p_values_matrix, k=1).T

    # Create a DataFrame with the matrix and league names as index/column names
    leagues = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame(p_values_matrix, index=leagues, columns=leagues)
    
    array = p_values.values
    np.fill_diagonal(array, np.nan)

    p_values = pd.DataFrame(array, columns=leagues, index=leagues)

    return p_values


sports_team_performance()